In [1]:
%%capture
!mamba install cudatoolkit xformers bitsandbytes pytorch pytorch-cuda=11.8 \
    -c pytorch -c nvidia -c xformers -c conda-forge -y




In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
!pip install "unsloth[kaggle] @ git+https://github.com/unslothai/unsloth.git"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-eqiucvyu/unsloth_6cfbdc2bf379440dbc0411a4d4b2c3fa
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-eqiucvyu/unsloth_6cfbdc2bf379440dbc0411a4d4b2c3fa
  Resolved https://github.com/unslothai/unsloth.git to commit 4606443b77f98a624896d4ca50710255d8436d86
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - \ done
  Installing backend dependencies ... - \ | done
  Preparing metadata (pyproject.toml) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/24

In [4]:
!pip install triton
!pip install xformers
!pip uninstall datasets -y
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
#### from unsloth import FastLanguageModel
import torch
from unsloth import FastLanguageModel
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-it-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

2024-04-14 16:44:52.421778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 16:44:52.421888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 16:44:52.520834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
from datasets import DatasetDict, Dataset, Features, Value
import pandas as pd
data = pd.read_excel("path to training dataset")
data.reset_index(inplace=True,drop=True)

In [12]:
def get_template(original,rewritten,info):
    template = f"""<start_of_turn>user
Hello Need Help . My Friend translated my original text into something new .\nOriginal Text : {original}\nRewritten Text : {rewritten}\nMy friend used a large language model to rewrite this . 
Can you craft a exact single-line prompt which he might have used against my original text to get to the rewritten text.<end_of_turn>
<start_of_turn>model\n
{info}"""
  return template

In [13]:
for index,value in enumerate(data['info']):
    value = value.replace("[","")
    value = value.replace("]","")
    value = value.replace(",","")
    value = value.replace("\n\n","")
    original = data.loc[index,'Original']
    rewritten = data.loc[index,'Rewritten']
    temp = get_template(original,rewritten,value)
    data.loc[index,'train_data'] = temp+"<eos>"

In [14]:
data = data[['train_data']]

In [17]:
train1 = data[:400]
test1 = data[400:]

In [18]:
features = Features({
    'train_data': Value('string')
})
train1= DatasetDict({"train" : Dataset.from_pandas(train1,features = features)})

features = Features({
    'train_data': Value('string')
})
test1= DatasetDict({"test" : Dataset.from_pandas(test1,features = features)})

In [19]:
tokenizer.padding_side='right'

'right'

In [20]:
import transformers
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train1['train'],
    eval_dataset = test1['test'],
    dataset_text_field = "train_data",
    dataset_num_proc = 2,
    packing = False,
    # Can make training 5x faster for short sequences.
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        eval_steps=10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False)
)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/96 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [21]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.391 GB of memory reserved.


In [22]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 400 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 200,015,872


Step,Training Loss
1,10.138300
2,10.328300
3,9.119200
4,9.151800
5,8.332300
6,6.921400
7,6.756500
8,5.494900
9,4.079900
10,4.218900


TrainOutput(global_step=50, training_loss=2.1921877896785738, metrics={'train_runtime': 494.1228, 'train_samples_per_second': 0.81, 'train_steps_per_second': 0.101, 'total_flos': 5963687748882432.0, 'train_loss': 2.1921877896785738, 'epoch': 1.0})

# Inference

In [23]:
import pandas as pd
sample = pd.read_csv("/kaggle/input/llm-prompt-recovery/train.csv")


In [24]:

original = sample.loc[0,'original_text']
rewritten = sample.loc[0,'rewritten_text']
template = f"""<start_of_turn>user
Hello Need Help . My Friend translated my original text into something new .\nOriginal Text : {original}\nRewritten Text : {rewritten}\nMy friend used a large language model to rewrite this . 
Can you craft a exact single-line prompt which he might have used against my original text to get to the rewritten text.<end_of_turn>
<start_of_turn>model\n"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[template
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256)
ans = tokenizer.batch_decode(outputs)[0]
start = ans.find("<start_of_turn>model")
ans = ans[start+20:]
print(ans.strip())

General Thought:I need to understand the original text and rewritten text.
Observation:I can Highlight key elements or changes in the rewritten text.
Action:Now I can use my above findings to craft the exact prompt.
Prompt:Rewrite the text in a more engaging and entertaining manner.<eos>


In [25]:
test = pd.read_excel("/kaggle/input/testda/data.xlsx")
test.reset_index(inplace=True,drop=True)
test.head(1)

,Original Text,Prompt,Rewritten Text
0,The sun rises in the east and sets in the west.,Rewrite this as a romantic poem.,"In the east, where dreams take flight, the sun..."


In [26]:
for index,value in enumerate(test['Prompt']):
    original = test.loc[index,'Original Text']
    rewritten = test.loc[index,'Rewritten Text']
    template = f"""<start_of_turn>user
Hello Need Help . My Friend translated my original text into something new .\nOriginal Text : {original}\nRewritten Text : {rewritten}\nMy friend used a large language model to rewrite this . 
Can you craft a exact single-line prompt which he might have used against my original text to get to the rewritten text.<end_of_turn>
<start_of_turn>model"""
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [template], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 256)
    outputs = tokenizer.batch_decode(outputs)[0]
    start = outputs.find("<start_of_turn>model")
    answer = outputs[start+20:]
    end = answer.find("<eos>")
    answer = answer[:end]
    strtt = answer.find("Prompt:")
    answer = answer[strtt+7:]
    print(answer)
    test.loc[index,'output'] = answer

Rewrite the text in a more metaphorical way, using imagery and symbolism to evoke the beauty of the sunrise and sunset.
Reconstruct this text as if it were a line from a spy thriller, where Einstein's theory is a cover for a secret mission.
Rewrite the text in a more formal tone, emphasizing the scientific accuracy and precision of the original statement.
Reconstruct this text as if it were a scene from a horror movie.
Rewrite the text in a more engaging and dramatic way.
Rewrite the text in a more engaging and imaginative way.
Rewrite the text in a more engaging and humorous tone.
Reconstruct this text as if it were a line from a philosophical journal written by Leonardo da Vinci himself.
Rewrite the text in a more formal tone, emphasizing the scientific nature of the statement.
Rewrite the text in a more engaging and captivating manner.
Rewrite the text in a more inspirational tone, emphasizing the power of courage, perseverance, and determination.
Rewrite the text in a more scientif

In [28]:
#Saving the adapter files 
model.save_pretrained("/kaggle/working/model_v1")